# Self-Driving Car Engineer Nanodegree


## Project: Advanced Lane Finding
***
This project improves upon the previous lane finding pipeline with advanced techniques to detect lane lines in a variety of lighting and shading conditions. The pipeline consists of the following steps:

1. **Camera Calibration**
    - In order to correct camera distortion in the test images and videos, camera calibration must first be performed using images of chessboard pattern. The resulting camera matrix and distortion coefficients can then be applied to correct for distortion in the test images and videos.

1. **Gradients and Color Thresholding**
    - A variety of gradient threshold and color thresholding algorithm can then be applied to the test images and videos. The goal is to process an image such that the lane lines can be easily isolated from other features in a binary image under a variety of lighting and shading conditions.

1. **Perspective Transform**
    - Using perspective transform, images and videos of the car's dashboard view can be transformed into bird's-eye view. This allows the warpped image to be further processed to identify the curvature of the road.
    
1. **Lane Finding**
    - With a bird's-eye view image of the road, lane markings can then be identified using sliding window algorithm or searching from previously identified lane markings.
    
1. **Curvature Measurement**
    - Once the lane markings have been identified, radius of curvature of the lane markings as well as the position of the vehicle with respect to the center (assuming the camera is mounted at the center of the car) can then be computed.
    
1. **Reverse Perspective Transform**
    - Once the lane markings have been identified, the identified lane lines can then be projected back onto the car's dashboard view via the perspective transform matrix computed in the previous steps.

### Import Packages

In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob
import os
%matplotlib inline

### Class Definition

In [2]:
# Define a class to receive the characteristics of each line detection
class Line():
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False  
        # how many iteration was the line not detected?
        self.detected_counter = 0  
        # maximum number of iteration that line not detected before performing sliding window search
        self.max_detected_counter = 25 
        # maximum number of x values to store from previous frame
        self.maxframe = 50
        # x values of the last n fits of the line
        self.recent_xfitted = [] 
        #average x values of the fitted line over the last n iterations
        self.bestx = None     
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = None  
        #polynomial coefficients for the most recent fit
        self.current_fit = [np.array([False])]  
        #radius of curvature of the line in some units
        self.radius_of_curvature = None 
        #distance in meters of vehicle center from the line
        self.line_base_pos = None 
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 
        #x values for detected line pixels
        self.allx = None  
        #y values for detected line pixels
        self.ally = None  

### Camera Calibration

In [3]:
# This function takes in a series of calibration image from camera_cal
# folder and compute the camera matrix and distortion coefficients.
#
# Input: none
#
# Output: mtx - camera matrix
#         dist - distortion coefficients

def CameraCalibration():
    # Prepare object points on a 9x6 chessboard pattern. Note that a 9x5
    # is chosen instead because findChessboardCorners() cannot detect
    # all 9x6 grid in some of the close-up images 
    nx = 9
    ny = 5
    
    # Arrays to store object points and image points from calibration images
    objpoints = [] # 3D points in real world space
    imgpoints = [] # 2D points in image plane
    
    # Prepare object points e.g. (0,0,0), (1,0,0)... (x,y,z), where z is 0
    objp = np.zeros((nx*ny,3), np.float32)
    objp[:,:2] = np.mgrid[0:nx,0:ny].T.reshape(-1,2)
    
    # Read a list of calibration images
    images = glob.glob('camera_cal/calibration*.jpg')
    
    for fname in images:
        img = mpimg.imread(fname)
    
        ## Convert to grayscale
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
        ## Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (nx,ny), None)
    
        # If corners are found, add object points and image points to the arrays
        if ret == True:
            imgpoints.append(corners)
            objpoints.append(objp)
    
            # Draw and display the corners
            #cv2.drawChessboardCorners(img, (nx, ny), corners, ret)
    
    # Compute camera matrix and distortion coefficients from object points
    # and image points arrays
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
        
    return mtx, dist

### Gradients Thresholding

In [4]:
# This function takes in an image and apply threshold on the grayscale
# gradient in either x or y direction 
#
# Input: img - image
#        orient - 'x' or 'y'
#        sobel_kernel - kernel size of the sobel operator
#        thresh - lower and higher bound for the gradient threshold
#
# Output: grad_binary - binary image that is gradient thresholded

def abs_sobel_thresh(img, orient='x', sobel_kernel=3, thresh=(0, 255)):
    # Apply the following steps to img
    # 1) Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # 2) Take the derivative in x or y given orient = 'x' or 'y'
    if orient == 'x':
        sobel = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    if orient == 'y':
        sobel = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # 3) Take the absolute value of the derivative or gradient
    abs_sobel = np.absolute(sobel)
    # 4) Scale to 8-bit (0 - 255) then convert to type = np.uint8
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    # 5) Create a mask of 1's where the scaled gradient magnitude 
            # is > thresh_min and < thresh_max
    sobel_binary = np.zeros_like(scaled_sobel)
    sobel_binary[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1
    # 6) Return this mask as your binary_output image
    grad_binary = sobel_binary
    return grad_binary

# This function takes in an image and apply threshold on the grayscale
# gradient in terms of magnitude of gradient
#
# Input: img - image
#        sobel_kernel - kernel size of the sobel operator
#        thresh - lower and higher bound for the gradient threshold
#
# Output: mag_binary - binary image that is gradient thresholded

def mag_thresh(image, sobel_kernel=3, mag_thresh=(0, 255)):
    # Apply the following steps to img
    # 1) Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    # 2) Take the gradient in x and y separately
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # 3) Calculate the magnitude
    sobelxy = (pow(sobelx, 2) + pow(sobely, 2))**0.5
    # 4) Scale to 8-bit (0 - 255) and convert to type = np.uint8
    scaled_sobel = np.uint8(255*sobelxy/np.max(sobelxy))
    # 5) Create a binary mask where mag thresholds are met
    sobel_binary = np.zeros_like(scaled_sobel)
    sobel_binary[(scaled_sobel >= mag_thresh[0]) & (scaled_sobel <= mag_thresh[1])] = 1
    # 6) Return this mask as your binary_output image
    mag_binary = sobel_binary
    return mag_binary

# This function takes in an image and apply threshold on the grayscale
# gradient in terms of direction of gradient
#
# Input: img - image
#        sobel_kernel - kernel size of the sobel operator
#        thresh - lower and higher bound for the gradient threshold
#
# Output: dir_binary - binary image that is gradient thresholded

def dir_threshold(image, sobel_kernel=3, thresh=(0, np.pi/2)):
    # Apply the following steps to img
    # 1) Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    # 2) Take the gradient in x and y separately
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # 3) Take the absolute value of the x and y gradients
    abs_sobelx = np.absolute(sobelx)
    abs_sobely = np.absolute(sobely)
    # 4) Use np.arctan2(abs_sobely, abs_sobelx) to calculate the direction of the gradient
    dir_sobel = np.arctan2(abs_sobely, abs_sobelx)
    # 5) Create a binary mask where direction thresholds are met
    sobel_binary = np.zeros_like(dir_sobel)
    sobel_binary[(dir_sobel >= thresh[0]) & (dir_sobel <= thresh[1])] = 1
    # 6) Return this mask as your binary_output image
    dir_binary = sobel_binary
    return dir_binary

### Colour Thresholding

In [5]:
# This function takes in an image and apply threshold on the grayscale
# gradient in terms of R-channel intensity in RGB color space
#
# Input: img - image
#        thresh - lower and higher bound for the color threshold
#
# Output: binary_output - binary image that is color thresholded
def rgb_select(img, thresh=(0,255)):
    # 1) Apply a threshold to the R channel
    # 2) Return a binary image of threshold result
    R = img[:,:,0]
    binary_output = np.zeros_like(R)
    binary_output[(R > thresh[0]) & (R <= thresh[1])] = 1
    return binary_output

# This function takes in an image and apply threshold on the grayscale
# gradient in terms of S-channel intensity in HSV color space
#
# Input: img - image
#        thresh - lower and higher bound for the color threshold
#
# Output: binary_output - binary image that is color thresholded
def hsv_select(img, thresh=(0,255)):
    # 1) Convert to HSV color space
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    H = hsv[:,:,0]
    S = hsv[:,:,1]
    V = hsv[:,:,2]
    # 2) Apply a threshold to the S channel
    # 3) Return a binary image of threshold result
    binary_output = np.zeros_like(S)
    binary_output[(S > thresh[0]) & (S <= thresh[1])] = 1
    return binary_output

# This function takes in an image and apply threshold on the grayscale
# gradient in terms of S-channel intensity in HLS color space
#
# Input: img - image
#        thresh - lower and higher bound for the color threshold
#
# Output: binary_output - binary image that is color thresholded
def hls_select(img, thresh=(0,255)):
    # 1) Convert to HLS color space
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    H = hls[:,:,0]
    L = hls[:,:,1]
    S = hls[:,:,2]
    # 2) Apply a threshold to the S channel
    # 3) Return a binary image of threshold result
    binary_output = np.zeros_like(S)
    binary_output[(S > thresh[0]) & (S <= thresh[1])] = 1
    return binary_output

### Perspective Transform

In [6]:
# The straight lines images in the "test_images" folder are used to define the
# source points for perspective transform. To do so, the images are first
# undistorted using the camera matrix and distortion coefficients computed in
# the Camera Calibration section. Then, the source points are handpicked in the
# images. Finally, a perspective transform is performed on those images, and
# straight lines are overlaid on top of the images to verify that the transform
# is working as expected.

# The source points are defined by manually locating the pixel of the bottom
# and top ends of the lane lines in the images. The x-coordinates of the
# source points are manually tuned by examining the straightness of the warped
# image
#
# straight_lines1
# (205, 720), (620, 435), (1105, 720), (655, 435)
# straight_lines2
# (220, 720), (620, 430), (1110, 720), (660, 435)

def warp(img):
    img_size = (img.shape[1], img.shape[0])
    # Four source coordinates
    src = np.float32(
        [[203, 720],
         [581, 460],
         [702, 460],
         [1099, 720]])
    # Four desired coordinates
    dst = np.float32(
        [[350, 720],
         [350, 0],
         [950, 0],
         [950, 720]])

    # Compute the perspective transform, M
    M = cv2.getPerspectiveTransform(src, dst)
    
    # Compute the inverse perspective transform, Minv
    Minv = cv2.getPerspectiveTransform(dst, src)
    
    # Create warped image - uses linear interpolation
    warped = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)#, borderMode=cv2.BORDER_REPLICATE)
    
    return M, Minv, warped

### Calculate Curvature

In [7]:
# This function takes in second-order polynomial coefficients
# and computes the radius of curvature as well as the distance
# from the vehicle center at the bottom of the image
#
# Input: left_fit - polynomial coefficients of fitted curve
#        right_fit - polynomial coefficients of fitted curve
#
# Output: left_pos_diff - distance of left curve from vehicle center (m)
#         right_pos_diff - distance of right curve from vehicle center (m)
#         left_curverad - radius of curvature of left curve (m)
#         right_curverad - radius of curvature of right curve (m)
def measure_curvature_real(left_fit, right_fit):
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/600 # meters per pixel in x dimension
    
    # Define y-value where we want radius of curvature and lane position
    # We'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = 720
    
    left_fit_cr = np.copy(left_fit)
    right_fit_cr = np.copy(right_fit)
    
    # Compute the distance between lane and vehicle center
    # Assuming the image center is aligned with vehicle center
    # Vehicle center pixel is 1280/2 = 640
    left_base_pos = left_fit_cr[0] * y_eval**2 + left_fit_cr[1] * y_eval + left_fit_cr[2]
    right_base_pos = right_fit_cr[0] * y_eval**2 + right_fit_cr[1] * y_eval + right_fit_cr[2]
    
    left_pos_diff = (left_base_pos - 640) * xm_per_pix
    right_pos_diff = (right_base_pos - 640) * xm_per_pix
    
    # Calculates the curvature of polynomial functions in meters.
    # Convert polynomial coefficient from pixel to m    
    left_fit_cr[0] *= (xm_per_pix / ym_per_pix**2)
    left_fit_cr[1] *= (xm_per_pix / ym_per_pix)
    
    right_fit_cr[0] *= (xm_per_pix / ym_per_pix**2)
    right_fit_cr[1] *= (xm_per_pix / ym_per_pix)
    
    # Calculation of R_curve (radius of curvature)
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
            
    return left_pos_diff, right_pos_diff, left_curverad, right_curverad

### Lane Finding

In [8]:
# This function takes in the x and y pixel of the identified
# lane marking and attempt to fit a second-order polynomial
# over them
#
# This function also performs sanity check on the fitted curve
# for the following:
# - left and right curve are separated by approximately the 
#   right distance horizontally
# - left and right curve are roughly parallel
#
# Input: img_shape - shapes of image
#        leftx - array of x pixel values of left lane
#        lefty - array of y pixel values of left lane
#        rightx - array of x pixel values of right lane
#        righty - array of y pixel values of right lane
#
# Output: left_fit - polynomial coefficients of fitted curve
#         right_fit - polynomial coefficients of fitted curve
#         left_fitx - array of x value of left curve
#         right_fitx - array of x value of right curve
#         ploty - array of y value of both curves
def fit_poly(img_shape, leftx, lefty, rightx, righty):
    # Fit a second order polynomial to each with np.polyfit()
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    # Generate x and y values for plotting
    ploty = np.linspace(0, img_shape[0]-1, img_shape[0])
    # Calc both polynomials using ploty, left_fit and right_fit
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        left_lane.detected = True
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        left_lane.detected = False
    try:
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
        right_lane.detected = True
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        right_fitx = 1*ploty**2 + 1*ploty
        right_lane.detected = False

    left_lane.current_fit = np.copy(left_fit)
    right_lane.current_fit = np.copy(right_fit)
    
    # Compute radius of curvature of left and right lanes of current frame
    left_pos_diff, right_pos_diff, left_curverad, right_curverad = \
        measure_curvature_real(left_lane.current_fit, right_lane.current_fit)

    ## Sanity check to make sure the fitted curves are correct
    ## 1 - checking for curvature is difficult and error prone in straight line
    #curvature_diff = np.absolute(left_curverad - right_curverad)
    #diff_perc = np.maximum(curvature_diff/left_curverad, curvature_diff/right_curverad)
    #if diff_perc > 0.5:
    #    left_lane.detected = False
    #    right_lane.detected = False
    # 2 - left and right curve are separated by approximately the 
    #   right distance horizontally
    lane_separation = right_pos_diff - left_pos_diff
    if lane_separation < 3.0 or lane_separation > 5.0:
        left_lane.detected = False
        right_lane.detected = False
    # 3 - checking for parallel lines is difficult and error prone in straight line    
    
    # Compute the averaged left and right curve fits over previous n iteration
    if left_lane.detected:
        # Check if maximum storage of x value of fitted line is reached
        # If so, remove the first element
        if len(left_lane.recent_xfitted) == left_lane.maxframe:
            left_lane.recent_xfitted.pop(0)
        left_lane.recent_xfitted.append(left_fitx)
        
        # Compute the average x value of fitted line over previous n iteration
        elemsum = np.zeros_like(left_lane.recent_xfitted[0])
        for element in left_lane.recent_xfitted:
            elemsum += element
        left_lane.bestx = elemsum / len(left_lane.recent_xfitted)
        
        # Compute the polynomial coefficients averaged over the last n iteration
        left_lane.best_fit = np.polyfit(ploty, left_lane.bestx, 2)

    if right_lane.detected:
        # Check if maximum storage of x value of fitted line is reached
        # If so, remove the first element
        if len(right_lane.recent_xfitted) == right_lane.maxframe:
            right_lane.recent_xfitted.pop(0)
        right_lane.recent_xfitted.append(right_fitx)
        
        # Compute the average x value of fitted line over previous n iteration
        elemsum = np.zeros_like(right_lane.recent_xfitted[0])
        for element in right_lane.recent_xfitted:
            elemsum += element
        right_lane.bestx = elemsum / len(right_lane.recent_xfitted)
        
        # Compute the polynomial coefficients averaged over the last n iteration
        right_lane.best_fit = np.polyfit(ploty, right_lane.bestx, 2)

    # Compute radius of curvature of averaged left and right curve fit
    if (left_lane.recent_xfitted or right_lane.recent_xfitted):
        left_pos_diff, right_pos_diff, left_curverad, right_curverad = \
            measure_curvature_real(left_lane.best_fit, right_lane.best_fit)

    left_lane.line_base_pos = np.copy(left_pos_diff)
    right_lane.line_base_pos = np.copy(right_pos_diff)
    left_lane.radius_of_curvature = np.copy(left_curverad)
    right_lane.radius_of_curvature = np.copy(right_curverad) 
    
    return left_fit, right_fit, left_fitx, right_fitx, ploty

# This function takes in an image and determine the location of
# the lane lines through finding peaks of histogram of the bottom
# half of binary image then performing sliding window search
# towards the top half of the image
#
# Input: binary_warped - image
#
# Output: left_fitx - array of x value of left curve
#         right_fitx - array of x value of right curve
#         ploty - array of y value of both curves
#         out_img - image
def sliding_search(binary_warped):
    # Grab only the bottom half of the image
    # Lane lines are likely to be mostly vertical nearest to the car
    bottom_half = binary_warped[binary_warped.shape[0]//2:,:]
    
    # Sum across image pixels vertically - make sure to set an `axis`
    # i.e. the highest areas of vertical lines should be larger values
    histogram = np.sum(bottom_half, axis=0)
    
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))

    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint
    
    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero (i.e. activated) pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        ### TO-DO: Find the four below boundaries of the window ###
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),
        (win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),
        (win_xright_high,win_y_high),(0,255,0), 2) 
        
        ### TO-DO: Identify the nonzero pixels in x and y within the window ###
        good_left_inds = ((nonzerox >= win_xleft_low) & (nonzerox <= win_xleft_high) &
                          (nonzeroy >= win_y_low) & (nonzeroy <= win_y_high)).nonzero()[0]
        good_right_inds = ((nonzerox >= win_xright_low) & (nonzerox <= win_xright_high) &
                           (nonzeroy >= win_y_low) & (nonzeroy <= win_y_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        ### TO-DO: If you found > minpix pixels, recenter next window ###
        ### (`right` or `leftx_current`) on their mean position ###
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    ## Visualization ##
    # Colors in the left and right lane regions
    out_img[lefty, leftx] = [255, 0, 0]
    out_img[righty, rightx] = [0, 0, 255]
    
    # Fit new polynomial
    left_fit, right_fit, left_fitx, right_fitx, ploty = fit_poly(binary_warped.shape, leftx, lefty, rightx, righty)
                           
    return left_fitx, right_fitx, ploty, out_img

# This function takes in an image and determine the location of
# the lane lines through searching the neighbourhood of the
# polynomial fitted to the left and right lane of the previous
# frame in the video
#
# Input: binary_warped - image
#
# Output: result - image
def search_around_poly(binary_warped):
    # HYPERPARAMETER
    # Choose the width of the margin around the previous polynomial to search
    # The quiz grader expects 100 here, but feel free to tune on your own!
    margin = 50

    # Grab activated pixels
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    left_fit = np.copy(left_lane.current_fit)
    right_fit = np.copy(right_lane.current_fit)
    
    ### Set the area of search based on activated x-values ###
    ### within the +/- margin of our polynomial function ###
    ### Hint: consider the window areas for the similarly named variables ###
    ### in the previous quiz, but change the windows to our new search area ###
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
                    left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
                    left_fit[1]*nonzeroy + left_fit[2] + margin)))
    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
                    right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
                    right_fit[1]*nonzeroy + right_fit[2] + margin)))
    
    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    # Fit new polynomials
    left_fit, right_fit, left_fitx, right_fitx, ploty = fit_poly(binary_warped.shape, leftx, lefty, rightx, righty)
    
    ## Visualization ##
    # Create an image to draw on and an image to show the selection window
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    window_img = np.zeros_like(out_img)
    # Color in left and right line pixels
    out_img[lefty, leftx] = [255, 0, 0]
    out_img[righty, rightx] = [0, 0, 255]

    # Generate a polygon to illustrate the search window area
    # And recast the x and y points into usable format for cv2.fillPoly()
    left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, 
                              ploty])))])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))
    right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
    right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, 
                              ploty])))])
    right_line_pts = np.hstack((right_line_window1, right_line_window2))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
    cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
    result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
    ## End visualization steps ##
    
    return left_fitx, right_fitx, ploty, result

def search_lane(binary_warped):    
    if (left_lane.detected == False) or (right_lane.detected == False):
        left_lane.detected_counter += 1
        
    # Start with sliding window search to identify the lane lines if the following is True
    # 1. left and right lanes haven't been detected for certain number of frames
    # 2. left and right lanes have never been fitted (start of video)
    # Else, perform a lane search using the fitted polynomial
    if (left_lane.detected_counter > left_lane.max_detected_counter) or \
       (right_lane.detected_counter > right_lane.max_detected_counter) or \
       (not left_lane.recent_xfitted or not right_lane.recent_xfitted):
        left_fitx, right_fitx, ploty, out_img = sliding_search(binary_warped)
    else:
        left_fitx, right_fitx, ploty, out_img = search_around_poly(binary_warped)
    if (left_lane.detected == True) and (right_lane.detected == True):
        left_lane.detected_counter = 0
        right_lane.detected_counter = 0
    return left_fitx, right_fitx, ploty, out_img 

## Test Image Pipeline

In [9]:
dirs = os.listdir("test_images/")

### Camera Calibration
mtx, dist = CameraCalibration()

for file in dirs:  
    # Instantiate left and right lane using Line class
    left_lane = Line()
    right_lane = Line()
    
    ### Undistort the image
    image = mpimg.imread("test_images/"+file)
    undist = cv2.undistort(image, mtx, dist, None, mtx)
    plt.imsave("output_images/"+"undist_"+os.path.basename(file), undist, format='png')
    
    ### Grandient Thresholding
    # Choose a Sobel kernel size
    ksize = 15 # Choose a larger odd number to smooth gradient measurements
    min_dir = 40 # Minimum lane angle from horizontal to be detected
    max_dir = 75 # Maximum lane angle from horizontal to be detected
    grad_low_threshold = 30 # Minimum grayscale gradient intensity to be detected
    grad_high_threshold = 100 # Maximum grayscale gradient intensity to be detected

    # Apply each of the thresholding functions
    gradx = abs_sobel_thresh(undist, orient='x', sobel_kernel=ksize, thresh=(grad_low_threshold, grad_high_threshold))

    grad_combined = np.zeros_like(gradx)
    grad_combined[(gradx == 1)] = 1
    plt.imsave("output_images/"+"grad_"+os.path.basename(file), grad_combined, format='png', cmap='gray')
    
    ### Color Thresholding
    hsv_low_threshold = 120 # Minimum satuation intensity to be detected
    hsv_high_threshold = 255 # Maximum satuation intensity to be detected

    hls_low_threshold = 120 # Minimum satuation intensity to be detected
    hls_high_threshold = 255 # Maximum satuation intensity to be detected

    rgb_low_threshold = 200 # Minimum R value to be detected
    rgb_high_threshold = 255 # Maximum R value to be detected

    # apply S-channel thresholding in HLS color space
    hls_binary = hls_select(undist, thresh=(hls_low_threshold, hls_high_threshold))
    
    # apply R-channel thresholding in RGB color space
    R_binary = rgb_select(undist, thresh=(rgb_low_threshold, rgb_high_threshold))
    
    color_binary = np.zeros_like(hls_binary)
    color_binary[(hls_binary == 1) & (R_binary == 1)] = 1
    plt.imsave("output_images/"+"color_"+os.path.basename(file), color_binary, format='png', cmap='gray')

    ### Combined Thresholding
    # combine color and gradient thresholded images
    combined_binary = np.zeros_like(color_binary)
    combined_binary[(color_binary == 1) | (grad_combined == 1)] = 1
    
    # color visualization of the contribution of color and gradient threshold
    visual_binary = np.dstack((color_binary, grad_combined, np.zeros_like(color_binary))) * 255
    plt.imsave("output_images/"+"combined_"+os.path.basename(file), visual_binary, format='png')
    
    ### Perspective Transform
    M, Minv, warped = warp(combined_binary)
    plt.imsave("output_images/"+"warped_"+os.path.basename(file), warped, format='png', cmap='gray')
    
    ### Lane Finding
    left_fitx, right_fitx, ploty, out_img = sliding_search(warped)
    
    # Generate a polygon to illustrate the fitted curve
    # And recast the x and y points into usable format for cv2.fillPoly()
    margin = 5
    left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, 
                              ploty])))])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))
    right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
    right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, 
                              ploty])))])
    right_line_pts = np.hstack((right_line_window1, right_line_window2))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(out_img, np.int_([left_line_pts]), (255 ,255, 0))
    cv2.fillPoly(out_img, np.int_([right_line_pts]), (255 ,255, 0))
    
    plt.imsave("output_images/"+"detected_"+os.path.basename(file), out_img, format='png')

    ### Drawing Lane on image
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    ### Inverse Perspective Transform
    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (image.shape[1], image.shape[0]))
    
    # Combine the result with the original image
    result = cv2.addWeighted(undist, 1, newwarp, 0.3, 0, dtype=cv2.CV_8U)
    
    ### Display lane information on image
    font                   = cv2.FONT_HERSHEY_SIMPLEX
    bottomLeftCornerOfText = (10,30)
    fontScale              = 1
    fontColor              = (255,255,255)
    lineType               = 2
    
    bottomLeftCornerOfText = (10,60)
    text = 'Lane Curvature: {:4.2f}'.format((left_lane.radius_of_curvature + right_lane.radius_of_curvature)/2)
    cv2.putText(result, text, bottomLeftCornerOfText, font, fontScale, fontColor, lineType)
    
    bottomLeftCornerOfText = (10,90)
    dist_to_center = right_lane.line_base_pos + left_lane.line_base_pos
    if dist_to_center >= 0.0:
        text = 'Vehicle is {:1.2f}m left of center'.format(np.absolute(dist_to_center))
    else:
        text = 'Vehicle is {:1.2f}m right of center'.format(np.absolute(dist_to_center))
    cv2.putText(result, text, bottomLeftCornerOfText, font, fontScale, fontColor, lineType)

    plt.imsave("output_images/"+"final_"+os.path.basename(file), result, format='png')

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


## Video Pipeline

In [10]:
def process_image(image):
    ### Undistort the image
    undist = cv2.undistort(image, mtx, dist, None, mtx)
    
    ### Grandient Thresholding
    # Choose a Sobel kernel size
    ksize = 15 # Choose a larger odd number to smooth gradient measurements
    min_dir = 40 # Minimum lane angle from horizontal to be detected
    max_dir = 75 # Maximum lane angle from horizontal to be detected
    grad_low_threshold = 30 # Minimum grayscale gradient intensity to be detected
    grad_high_threshold = 100 # Maximum grayscale gradient intensity to be detected

    # Apply each of the thresholding functions
    gradx = abs_sobel_thresh(undist, orient='x', sobel_kernel=ksize, thresh=(grad_low_threshold, grad_high_threshold))

    grad_combined = np.zeros_like(gradx)
    grad_combined[(gradx == 1)] = 1
    
    ### Color Thresholding
    hsv_low_threshold = 120 # Minimum satuation intensity to be detected
    hsv_high_threshold = 255 # Maximum satuation intensity to be detected

    hls_low_threshold = 120 # Minimum satuation intensity to be detected
    hls_high_threshold = 255 # Maximum satuation intensity to be detected

    rgb_low_threshold = 200 # Minimum R value to be detected
    rgb_high_threshold = 255 # Maximum R value to be detected

    # apply S-channel thresholding in HLS color space
    hls_binary = hls_select(undist, thresh=(hls_low_threshold, hls_high_threshold))
    
    # apply R-channel thresholding in RGB color space
    R_binary = rgb_select(undist, thresh=(rgb_low_threshold, rgb_high_threshold))
    
    color_binary = np.zeros_like(hls_binary)
    color_binary[(hls_binary == 1) & (R_binary == 1)] = 1

    ### Combined Thresholding
    # combine color and gradient thresholded images
    combined_binary = np.zeros_like(color_binary)
    combined_binary[(color_binary == 1) | (grad_combined == 1)] = 1
    
    ### Perspective Transform
    M, Minv, warped = warp(combined_binary)
    
    ### Lane Finding
    left_fitx, right_fitx, ploty, out_img = search_lane(warped)

    ### Drawing Lane on image
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (image.shape[1], image.shape[0]))
    
    # Combine the result with the original image
    result = cv2.addWeighted(undist, 1, newwarp, 0.3, 0, dtype=cv2.CV_8U)

    ### Display lane information on image
    font                   = cv2.FONT_HERSHEY_SIMPLEX
    bottomLeftCornerOfText = (10,30)
    fontScale              = 1
    fontColor              = (255,255,255)
    lineType               = 2
    
    bottomLeftCornerOfText = (10,60)
    text = 'Lane Curvature: {:4.2f}'.format((left_lane.radius_of_curvature + right_lane.radius_of_curvature)/2)
    cv2.putText(result, text, bottomLeftCornerOfText, font, fontScale, fontColor, lineType)
    
    bottomLeftCornerOfText = (10,90)
    dist_to_center = right_lane.line_base_pos + left_lane.line_base_pos
    if dist_to_center >= 0.0:
        text = 'Vehicle is {:1.2f}m left of center'.format(np.absolute(dist_to_center))
    else:
        text = 'Vehicle is {:1.2f}m right of center'.format(np.absolute(dist_to_center))
    cv2.putText(result, text, bottomLeftCornerOfText, font, fontScale, fontColor, lineType)
    
    return result

def process_grad(image):
    ### Undistort the image
    undist = cv2.undistort(image, mtx, dist, None, mtx)
    
    ### Grandient Thresholding
    # Choose a Sobel kernel size
    ksize = 15 # Choose a larger odd number to smooth gradient measurements
    min_dir = 40 # Minimum lane angle from horizontal to be detected
    max_dir = 75 # Maximum lane angle from horizontal to be detected
    grad_low_threshold = 30 # Minimum grayscale gradient intensity to be detected
    grad_high_threshold = 100 # Maximum grayscale gradient intensity to be detected

    # Apply each of the thresholding functions
    gradx = abs_sobel_thresh(undist, orient='x', sobel_kernel=ksize, thresh=(grad_low_threshold, grad_high_threshold))

    grad_combined = np.zeros_like(gradx)
    grad_combined[(gradx == 1)] = 1
    
    ### Color Thresholding
    hsv_low_threshold = 120 # Minimum satuation intensity to be detected
    hsv_high_threshold = 255 # Maximum satuation intensity to be detected

    hls_low_threshold = 120 # Minimum satuation intensity to be detected
    hls_high_threshold = 255 # Maximum satuation intensity to be detected

    rgb_low_threshold = 200 # Minimum R value to be detected
    rgb_high_threshold = 255 # Maximum R value to be detected

    # apply S-channel thresholding in HLS color space
    hls_binary = hls_select(undist, thresh=(hls_low_threshold, hls_high_threshold))
    
    # apply R-channel thresholding in RGB color space
    R_binary = rgb_select(undist, thresh=(rgb_low_threshold, rgb_high_threshold))
    
    color_binary = np.zeros_like(hls_binary)
    color_binary[(hls_binary == 1) & (R_binary == 1)] = 1

    ### Combined Thresholding
    # combine color and gradient thresholded images
    combined_binary = np.zeros_like(color_binary)
    combined_binary[(color_binary == 1) | (grad_combined == 1)] = 1
    
    ### Perspective Transform
    M, Minv, warped = warp(combined_binary)
    
    ### Lane Finding
    left_fitx, right_fitx, ploty, out_img = search_lane(warped)

    ### Drawing Lane on image
    # Create an image to draw the lines on
    grad_zero = np.zeros_like(combined_binary).astype(np.uint8)
    grad_warp = np.dstack((grad_combined, color_binary, grad_zero)) * 255

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(out_img, Minv, (image.shape[1], image.shape[0]))
    
    # Combine the result with the original image
    result = cv2.addWeighted(grad_warp, 1, newwarp, 1, 0, dtype=cv2.CV_8U)

    ### Display lane information on image
    font                   = cv2.FONT_HERSHEY_SIMPLEX
    bottomLeftCornerOfText = (10,30)
    fontScale              = 1
    fontColor              = (255,255,255)
    lineType               = 2
    
    text = 'Left Lane Detected: {}'.format(left_lane.detected)
    cv2.putText(result, text, bottomLeftCornerOfText, font, fontScale, fontColor, lineType)
    
    bottomLeftCornerOfText = (10,60)
    text = 'Left Lane Fit Coef: '
    for element in left_lane.current_fit:
        text += '{:2.6f} '.format(element)
    cv2.putText(result, text, bottomLeftCornerOfText, font, fontScale, fontColor, lineType)
    
    bottomLeftCornerOfText = (10,90)
    text = 'Left Lane Curvature: {:4.2f}'.format(left_lane.radius_of_curvature)
    cv2.putText(result, text, bottomLeftCornerOfText, font, fontScale, fontColor, lineType)
    
    bottomLeftCornerOfText = (10,120)
    text = 'Right Lane Detected: {}'.format(right_lane.detected)
    cv2.putText(result, text, bottomLeftCornerOfText, font, fontScale, fontColor, lineType)
    
    bottomLeftCornerOfText = (10,150)
    text = 'Right Lane Fit Coef: '
    for element in right_lane.current_fit:
        text += '{:2.6f} '.format(element)
    cv2.putText(result, text, bottomLeftCornerOfText, font, fontScale, fontColor, lineType)
    
    bottomLeftCornerOfText = (10,180)
    text = 'Right Lane Curvature: {:4.2f}'.format(right_lane.radius_of_curvature)
    cv2.putText(result, text, bottomLeftCornerOfText, font, fontScale, fontColor, lineType)
    
    bottomLeftCornerOfText = (10,210)
    dist_to_center = right_lane.line_base_pos + left_lane.line_base_pos
    if dist_to_center >= 0.0:
        text = 'Vehicle is {:1.2f}m left of center'.format(np.absolute(dist_to_center))
    else:
        text = 'Vehicle is {:1.2f}m right of center'.format(np.absolute(dist_to_center))
    cv2.putText(result, text, bottomLeftCornerOfText, font, fontScale, fontColor, lineType)
    
    return result

## Test on Video

In [11]:
### Camera Calibration
mtx, dist = CameraCalibration()

# Instantiate left and right lane using Line class
left_lane = Line()
right_lane = Line()

# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

white_output = 'project_video_output.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,1)
clip1 = VideoFileClip("project_video.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

t:   0%|▏                                                                   | 3/1260 [00:00<00:59, 21.18it/s, now=None]

Moviepy - Building video project_video_output.mp4.
Moviepy - Writing video project_video_output.mp4



Moviepy - Done !
Moviepy - video ready project_video_output.mp4
Wall time: 1min 31s
